In [1]:
from tqdm import tqdm
import pickle

from skimage import io, measure
import numpy as np 
import h5py
from brainlit.preprocessing import image_process
from brainlit.algorithms.connect_fragments import dynamic_programming_viterbi2
import scipy.ndimage as ndi 
from sklearn.metrics import pairwise_distances_argmin_min
import napari
import networkx as nx

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [14]:
num = 9

res = [0.3,0.3,1]

# read image
im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_" + str(num) + "_first10_quantitative.tif"
im_og = io.imread(im_path, plugin="tifffile")
print(f"Image shape: {im_og.shape}")

# read coords
csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/voxel_coords.csv"

coords = np.genfromtxt(csv_path, delimiter=',')
coords = coords[num*10:(num+1)*10].astype(int)
coords_list = list(coords)
coords_list = [list(c) for c in coords_list]
coords_list.reverse()
print(f"coords shape: {coords.shape}")

# read ilastik
pred_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/2018-08-01_" + str(num) + "_first10_quantitative_Probabilities.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

threshold = 0.9  # 0.1
labels = measure.label(im_processed > threshold)

Image shape: (259, 443, 112)
coords shape: (10, 3)


In [15]:
soma_coords = [list(coords[0,:])]
axon_coords = [list(coords[-1,:])]

if num == 0:
    soma_coords += [[226, 643, 92], [233, 459, 120], [439, 500, 62]]
elif num == 7:
    soma_coords += [[335, 178, 39], [1, 418, 322]]

# Process Labels

In [16]:
new_labels = image_process.split_frags(soma_coords, labels, im_processed, threshold, res)

100%|██████████| 793/793 [00:02<00:00, 308.96it/s] 
286it [00:17, 16.13it/s]                         


In [ ]:
viewer = napari.Viewer(ndisplay=3)

viewer.add_image(im_og)
viewer.add_labels(new_labels)

In [18]:
_, axon_lbls = image_process.label_points(new_labels, axon_coords, res)
_, soma_lbls = image_process.label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]

print(f"Axon label: {axon_lbls}, soma labels: {soma_lbls}")

Axon label: [431], soma labels: [43]


In [19]:
viterbi = dynamic_programming_viterbi2.most_probable_neuron_path(image=im_og.astype(float), labels=new_labels, soma_lbls=soma_lbls, resolution=(0.3, 0.3, 1), coef_dist=10, coef_curv=1000)
viterbi.frags_to_lines()

Computing line representations: 100%|██████████| 507/507 [00:08<00:00, 60.89it/s]


In [20]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/mpnp/objects/viterbi_'+str(num)+'.pickle', 'wb') as handle:
    pickle.dump(viterbi, handle)

In [ ]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/mpnp/objects/viterbi_'+str(num)+'.pickle', 'rb') as handle:
    viterbi = pickle.load(handle)

im = viterbi.image
im_og = im
new_labels = viterbi.labels
soma_lbls = viterbi.soma_lbls

_, axon_lbls = image_process.label_points(new_labels, axon_coords, res)
_, soma_lbls = image_process.label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]
print(f"Axon labels: {axon_lbls}, soma labels: {soma_lbls}")

In [21]:
viterbi.reset_dists(type="all")
viterbi.compute_all_costs_dist(point_point_func=viterbi.point_point_dist, point_blob_func=viterbi.point_blob_dist)
viterbi.compute_all_costs_int()

Computing state costs (intensity): 100%|██████████| 1026169/1026169 [00:04<00:00, 206802.11it/s]


In [22]:
viterbi.create_nx_graph()

Adding edges to nx graph: 100%|██████████| 1013/1013 [00:03<00:00, 302.33it/s]


In [23]:
axon_lbl= axon_lbls[0]

start1 = viterbi.comp_to_states[axon_lbl][0]
start2 = viterbi.comp_to_states[axon_lbl][1]
end_state = viterbi.comp_to_states[soma_lbl][0]

if num in [1, 3, 4, 6, 7, 8]:
    start = start1
elif num in [0, 2, 5, 9]:
    start = start2

In [24]:
path_states = nx.shortest_path(viterbi.nxGraph, start, end_state, weight='weight')

In [18]:
path_comps = []
for state in path_states:
    path_comps.append(viterbi.state_to_comp[state][1])
print(f'path sequence: {path_states}')
print(f'component sequence: {path_comps}')

path_mask = 0*new_labels
for i, label in enumerate(path_comps):
    path_mask[new_labels == label] = i+1

soma_mask = 0*new_labels
for soma_lbl in viterbi.soma_lbls:
    soma_mask[new_labels == soma_lbl] = soma_lbl

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(viterbi.image)
viewer.add_labels(new_labels)
viewer.add_labels(path_mask)
viewer.add_labels(soma_mask)
viewer.add_labels(new_labels == axon_lbl)

viewer.add_points([axon_coords[0]], face_color="red", size=10)

lines = []
cumul_cost = 0
for s, state in enumerate(path_states):
    if s>0:
        dist_cost = viterbi.cost_mat_dist[path_states[s-1], state]
        int_cost = viterbi.cost_mat_int[path_states[s-1], state]
        cumul_cost += dist_cost + int_cost
        print(f"Trans. #{s}: dist cost state {path_states[s-1]}->state {state}, comp {viterbi.state_to_comp[path_states[s-1]][1]}->comp {viterbi.state_to_comp[state][1]}: {dist_cost:.2f}, int cost: {int_cost:.2f}, cum. cost: {cumul_cost:.2f}")
    if viterbi.state_to_comp[state][0] == "fragment":
        lines.append(list(viterbi.state_to_comp[state][2]["coord1"]))
        lines.append(list(viterbi.state_to_comp[state][2]["coord2"]))
    elif viterbi.state_to_comp[path_states[s-1]][0] == "fragment":
        lines.append(list(viterbi.state_to_comp[path_states[s-1]][2]["soma connection point"]))
lines.insert(0, coords_list[0]) 
lines.append(coords_list[-1])      
viewer.add_shapes(lines, shape_type="path", edge_color="blue", edge_width=2)
viewer.add_shapes(coords_list, shape_type="path", edge_color="green", edge_width=2)

viewer.camera.angles = [0, -90, 180]
napari.run()


NameError: name 'viterbi' is not defined